# Practice: Independent One Factor Design

In [ ]:
# Enable the commands below when running this program on Google Colab.
# !pip install arviz==0.7
# !pip install pymc3==3.8
# !pip install Theano==1.0.4

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm

plt.style.use('seaborn-darkgrid')
np.set_printoptions(precision=3)
pd.set_option('display.precision', 3)

In [ ]:
LD_GROUP = [5.02, 6.67, 8.17, 2.79, 8.13, 6.34, 6.32, 3.97]
LL_GROUP = [9.89, 9.58, 11.20, 9.05, 12.33, 9.39, 10.88, 9.37, 17.40]
DM_GROUP = [10.20, 7.29, 7.57, 3.42, 5.82, 10.92, 5.21, 13.47, 8.64, 6.05]
levels = ['LD group', 'LL group', 'DM group']

In [ ]:
# Data visualization
plt.boxplot([LD_GROUP, LL_GROUP, DM_GROUP], labels=levels)
plt.ylabel('Weight')
plt.show()

In [ ]:
# Summary
data = pd.DataFrame([LD_GROUP, LL_GROUP, DM_GROUP], index=levels).transpose()
data.describe()

In [ ]:
# Unbalanced data, observed data should be formatted
observed = np.concatenate([LD_GROUP, LL_GROUP, DM_GROUP])
print(observed)
idx = [0] * len(LD_GROUP) + [1] * len(LL_GROUP) + [2] * len(DM_GROUP)
print(idx)

## Bayesian analysis

In [ ]:
with pm.Model() as model:
    # Prior distribution
    mu = pm.Uniform('mu', 0, 20, shape=len(levels))
    sigma = pm.Uniform('sigma', 0, 10)

    # Likelihood
    y_pred = pm.Normal('y_pred', mu=mu[idx], sigma=sigma, observed=observed)

    # Total mean
    total_mean = pm.Deterministic('total_mean', (pm.math.sum(mu) / len(levels)))

    # Effect of each level
    a = pm.Deterministic('a', mu - total_mean)

    # Variance of factor (season)
    sigma_factor = pm.Deterministic('sigma_factor',
                        pm.math.sqrt(pm.math.sum(a**2) / len(levels)))
    
    # Coefficient of determination
    eta_square = pm.Deterministic('eta_square', sigma_factor**2 / (sigma_factor**2 + sigma**2))  

    # Effect size
    delta = pm.Deterministic('delta', sigma_factor / sigma)

    # Post analysis
    mu2_mu1 = pm.Deterministic('mu2 - mu1', mu[1] - mu[0])
    mu2_mu3 = pm.Deterministic('mu2 - mu3', mu[1] - mu[2])

    delta_21 = pm.Deterministic('delta_21', mu2_mu1 / sigma)  # effect size
    delta_23 = pm.Deterministic('delta_23', mu2_mu3 / sigma)  # effect size

    trace = pm.sample(21000, chains=5)

In [ ]:
chain = trace[1000:]
pm.traceplot(chain)
plt.show()

In [ ]:
pm.summary(chain)

In [ ]:
plt.boxplot(
    [chain['a'][:,i] for i in range(len(levels))],
    labels=levels)
plt.ylim(-5, 6)
plt.xlabel('Effect of level')
plt.show()

### 水準の効果の有無
基準(0)より大きいか小さいか

In [ ]:
print('-- a_j > 0 --')
for i in range(len(levels)):
    print('{}: {:.3f} %'.format(levels[i], (chain['a'][:,i] > 0).mean() * 100))

print()

print('-- a_j < 0 --')
for i in range(len(levels)):
    print('{}: {:.3f} %'.format(levels[i], (chain['a'][:,i] < 0).mean() * 100))

### 要因の効果の大きさ

In [ ]:
pm.plot_posterior(chain['eta_square'], credible_interval=0.95, point_estimate='mode')
plt.xlabel('Coefficient of determination (CoD)')

pm.plot_posterior(chain['delta'], credible_interval=0.95, point_estimate='mode')
plt.xlabel('Effect size')

plt.show()

In [ ]:
print('Effect (SD) of Factor A: {:.3f} ({:.3f}) [{:.3f}, {:.3f}] = {:.1f} g'.format(chain['sigma_factor'].mean(), chain['sigma_factor'].std(), np.quantile(chain['sigma_factor'], 0.025), np.quantile(chain['sigma_factor'], 0.975), chain['sigma_factor'].mean()))

# if CoD = 0 (0%) -> The factor does not explain the observed data at all.
# if CoD = 1 (100%)  -> The factor well explains the observed data.
print('CoD: {:.3f} ({:.3f}) [{:.3f}, {:.3f}] = {:.1f} %'.format(chain['eta_square'].mean(), chain['eta_square'].std(), np.quantile(chain['eta_square'], 0.025), np.quantile(chain['eta_square'], 0.975), chain['eta_square'].mean() * 100))

print('Effect size: {:.3f} ({:.3f}) [{:.3f}, {:.3f}] = {:.1f} %'.format(chain['delta'].mean(), chain['delta'].std(), np.quantile(chain['delta'], 0.025), np.quantile(chain['delta'], 0.975), chain['delta'].mean() * 100))

### 水準間の比較

In [ ]:
def compare(a, b):
    return (chain['mu'][:,a] - chain['mu'][:,b] > 0).mean()

In [ ]:
# 行iの水準が列jの水準より大きい確率
result = pd.DataFrame(
    [[0, compare(0, 1), compare(0, 2)],
     [compare(1, 0), 0, compare(1, 2)],
     [compare(2, 0), compare(2, 1), 0]
    ],
    columns=levels,
    index=levels)
display(result)
# 95%以上の確率であると「別々に」明言できるのは、
# LL group > LD group
# LL group > DM group
# 「別々に」：同時に成り立つ確率は異なるため

### 特に興味のある2水準間の比較

In [ ]:
print('Mouse weight in LL group is average {:.2f} g ({:.3f}) [{:.3f}, {:.3f}] greater than in LD group.'.format(chain['mu2 - mu1'].mean(), chain['mu2 - mu1'].std(), np.quantile(chain['mu2 - mu1'], 0.025), np.quantile(chain['mu2 - mu1'], 0.975)))
print('Mouse weight in LL group is average {:.2f} g ({:.3f}) [{:.3f}, {:.3f}] greater than in DM group.'.format(chain['mu2 - mu3'].mean(), chain['mu2 - mu3'].std(), np.quantile(chain['mu2 - mu3'], 0.025), np.quantile(chain['mu2 - mu3'], 0.975)))